In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

from uuid6 import UUID

from insync.db import ListDB
from insync.list import CompletionCommand, Context, ContextType, ListItem, ListRegistry


In [ ]:
descs = ['eggs', 'milk', 'bread', 'butter']

reg = ListRegistry()

for desc in descs:
    i = ListItem(desc, context=Context('grocery', ContextType.CHECKLIST))
    reg.add(i)


ids = [li.uuid for li in reg.items]

def test_completion(uuid: UUID, completed: bool) -> None:
    com = CompletionCommand(uuid, completed)
    print(com)
    reg.do(com)
    print('  ', com)
    print(reg)

def test_undo() -> None:
    com = next(iter(reg.undo_stack))
    print(com)
    reg.undo(com)
    print('  ',com)
    print(reg)

test_completion(ids[0], True)
test_undo()

DB_FILE = Path("scratch.db")
DB_FILE.unlink(missing_ok=True)
db = ListDB(DB_FILE)
db.patch(reg)
test_completion(ids[2], True)

In [ ]:
db.patch(reg)

In [ ]:
# read in from the db
reg2 = db.load()

print(reg)
print(reg2)